# Generating new digital images  
This notebook is used to create new "digital gas pump display" pictures based on templates extracted from the original set of pictures.

In [0]:
# mounting on Google Drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
from PIL import Image, ImageDraw, ImageFont, ImageOps
from imgaug import augmenters as iaa
from matplotlib.pyplot import imshow

import numpy as np
import cv2
import glob
import string
import ntpath
import os
import uuid
import csv

%matplotlib inline

In [0]:
# defining augmentation sequence

seq = iaa.Sequential([
    iaa.CropAndPad(percent=(-0.10, 0.10)),
    iaa.AverageBlur(k=(2, 25)),
    iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)),
    iaa.Sometimes(0.25, iaa.CoarseDropout((0.0, 0.05), size_percent=(0.2, 0.25))),
    iaa.Affine(rotate=(-12,12),
               shear=(-15,15)),
    iaa.Add((-10,10), per_channel=0.5)
], random_order=True)

In [0]:
def generate_images(n_generations, template, output_dir):
  
  # generate 'n_generations' random five-digit numbers
  numlst = []

  for i in range(n_generations):
    num = []
    for j in range(5):
      dig = np.random.randint(1,10)
      num.append(dig)
    numlst.append(num)
  
  if template == 1:
    font_size = 240
  else:
    font_size = 130
    
  shadowcolor=(20,20,20)
  fillcolor=(20,20,20)
  font = ImageFont.truetype('/content/drive/My Drive/Team Bergere/Optimizer_scripts/end2end_method/image_creation/fonts/team_bergere.ttf', font_size)
  
  labels = []
  
  # generate a new image for each generated number  
  for num in numlst:
    
    # load template 
    img = Image.open("/content/drive/My Drive/Team Bergere/Optimizer_scripts/end2end_method/image_creation/templates/template{}.jpg".format(template))
    
    # generate different combinations of number of digits and number of decimals 
    p = np.random.uniform()

    # 5 digits, 2 decimal in 45% of the cases
    if p > 0.55:
      text = "{}{}{}.{}{}".format(num[0],num[1],num[2],num[3],num[4])
      
      if template == 1:
        x = 600
        y = 515
      else:
        x = 775
        y = 235

    # 4 digits, 2 decimal in 45% of the cases
    elif p > 0.10:
      text = "{}{}.{}{}".format(num[0],num[1],num[2],num[3])
      
      if template == 1:
        x = 760
        y = 515
      else:
        x = 865
        y = 235

    # 4 digits, 1 decimal in 10% of the cases
    else: 
      text = "{}{}{}.{}".format(num[0],num[1],num[2],num[3])
      
      if template == 1:
        x = 600
        y = 515
      else:
        x = 775
        y = 235

    draw = ImageDraw.Draw(img)

    # draw text (number) borders
    draw.text((x-1, y-1), text, font=font, fill=shadowcolor)
    draw.text((x+1, y-1), text, font=font, fill=shadowcolor)
    draw.text((x-1, y+1), text, font=font, fill=shadowcolor)
    draw.text((x+1, y+1), text, font=font, fill=shadowcolor)

    # draw text (number) fill
    draw.text((x, y), text, font=font, fill=fillcolor)

    # augment image
    img = np.array(img)
    aug = seq(images=[img])
    aug = aug[0]

    # save image to Drive 
    fname = str(uuid.uuid4()) # create unique filename
    cv2.imwrite(output_dir + fname + ".jpg", aug) 

    # save label 
    label = (fname, text)
    labels.append(label)
    
  # write all labels to csv
  with open(output_dir + "labels_template{}.csv".format(template),'w') as out:
      csv_out = csv.writer(out)
      csv_out.writerow(['filename','label'])
      for row in labels:
          csv_out.writerow(row)

In [0]:
# generate_images(n_generations=10000, template=1, output_dir="/content/drive/My Drive/Team Bergere/Optimizer_scripts/end2end_method/image_creation/test/")
# generate_images(n_generations=10000, template=2, output_dir="/content/drive/My Drive/Team Bergere/Optimizer_scripts/end2end_method/image_creation/test/")